In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

# Verileri oku
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

# Hedef değişken
y = train['SalePrice']

# ID'leri sakla
test_ids = test['Id']

# 'SalePrice' ve 'Id' dışındaki her şeyi birleştir
combine = pd.concat([train.drop(['SalePrice'], axis=1), test], sort=False).reset_index(drop=True)

# Kategorik değişkenleri label encode et
categorical = combine.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical:
    le = LabelEncoder()
    combine[col] = combine[col].fillna("Missing")
    combine[col] = le.fit_transform(combine[col])
    label_encoders[col] = le

# Sayısal eksik değerleri median ile doldur
numeric = combine.select_dtypes(include=[np.number]).columns
for col in numeric:
    combine[col] = combine[col].fillna(combine[col].median())

# Özellikler (hepsi)
features = combine.columns

# Train/test ayrımı
X_train = combine.iloc[:len(train), :]
X_test = combine.iloc[len(train):, :]

# Model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(X_train, y)

# Performans kontrolü
scores = cross_val_score(model, X_train, y, scoring='neg_root_mean_squared_error', cv=5)
print(f"CV RMSE: {-scores.mean():.2f}")

# Tahmin ve gönderim
predictions = model.predict(X_test)
submission = pd.DataFrame({
    'Id': test_ids,
    'SalePrice': predictions
})
submission.to_csv('submission.csv', index=False)


CV RMSE: 29820.06
